In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


# Data Loading

In [2]:
train_data= pd.read_csv("../input/students-drop-out-prediction/train.csv")
test_data = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [3]:
train_data.head()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40,label
0,0,3,6,11,55,1,1,16,5,60,...,-0.3,0.79,1,93,47,1,48,26,0,1
1,1,8,94,12,65,1,4,22,4,173,...,5.4,6.51,6,29,37,25,53,35,7,2
2,2,3,166,10,155,1,1,16,1,309,...,0.6,2.02,1,8,34,4,43,38,2,1
3,3,3,6,13,121,1,1,16,1,6,...,0.6,2.02,7,37,40,27,43,65,0,1
4,4,4,51,18,21,1,17,22,118,168,...,8.4,12.51,16,22,25,25,24,23,1,1


In [4]:
print(train_data.shape)
print(train_data.columns)

(3796, 43)
Index(['id', 'v_21', 'v_32', 'v_15', 'v_4', 'v_1', 'v_8', 'v_12', 'v_25',
       'v_20', 'v_18', 'v_3', 'v_11', 'v_14', 'v_26', 'v_27', 'v_30', 'v_9',
       'v_31', 'v_28', 'v_13', 'v_33', 'v_17', 'v_19', 'v_29', 'v_23', 'v_24',
       'v_10', 'v_5', 'v_6', 'v_22', 'v_0', 'v_7', 'v_2', 'v_16', 'v_34',
       'v_35', 'v_36', 'v_37', 'v_38', 'v_39', 'v_40', 'label'],
      dtype='object')


In [5]:
print(test_data.shape)

(1628, 42)


In [6]:
train_data.isnull().sum()   # Checking null values in the data set

id       0
v_21     0
v_32     0
v_15     0
v_4      0
v_1      0
v_8      0
v_12     0
v_25     0
v_20     0
v_18     0
v_3      0
v_11     0
v_14     0
v_26     0
v_27     0
v_30     0
v_9      0
v_31     0
v_28     0
v_13     0
v_33     0
v_17     0
v_19     0
v_29     0
v_23     0
v_24     0
v_10     0
v_5      0
v_6      0
v_22     0
v_0      0
v_7      0
v_2      0
v_16     0
v_34     0
v_35     0
v_36     0
v_37     0
v_38     0
v_39     0
v_40     0
label    0
dtype: int64

In [7]:
train_data.label.value_counts()

1    1863
0    1243
2     690
Name: label, dtype: int64

In [8]:
x_train = train_data.drop(columns='label')
y_label = train_data['label']

In [9]:
x_train.describe()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_7,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40
count,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,...,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000,3796.000000
mean,1897.500000,4.817439,70.825606,11.609062,99.329294,0.911486,3.436249,18.970495,97.203372,166.138567,...,12.394810,2.041333,0.835901,5.341149,50.674658,26.212065,20.837197,31.276607,40.210485,3.278978
std,1095.955139,6.166860,56.110329,2.576362,45.017380,0.284079,4.577644,15.010042,79.213308,103.121322,...,3.414303,2.516779,3.109413,3.507388,29.330820,14.929649,11.984063,17.847942,22.466458,2.618851
min,0.000000,3.000000,6.000000,9.000000,9.000000,0.000000,1.000000,16.000000,1.000000,6.000000,...,7.600000,-0.800000,-4.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,948.750000,3.000000,6.000000,10.000000,55.000000,1.000000,1.000000,16.000000,5.000000,60.000000,...,9.400000,0.500000,-1.700000,3.000000,25.000000,13.000000,11.000000,16.000000,21.750000,1.000000
50%,1897.500000,3.000000,89.000000,10.000000,101.000000,1.000000,1.000000,16.000000,110.000000,168.000000,...,12.400000,1.400000,0.790000,5.000000,50.000000,26.000000,21.000000,31.000000,41.000000,3.000000
75%,2846.250000,4.000000,116.000000,12.000000,146.000000,1.000000,3.000000,16.000000,181.000000,251.000000,...,14.900000,2.800000,2.020000,8.000000,76.000000,39.000000,31.000000,47.000000,59.000000,5.000000
max,3795.000000,61.000000,176.000000,25.000000,176.000000,1.000000,25.000000,179.000000,289.000000,322.000000,...,25.200000,12.700000,12.510000,18.000000,107.000000,58.000000,49.000000,69.000000,87.000000,14.000000


In [10]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3796 entries, 0 to 3795
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3796 non-null   int64  
 1   v_21    3796 non-null   int64  
 2   v_32    3796 non-null   int64  
 3   v_15    3796 non-null   int64  
 4   v_4     3796 non-null   int64  
 5   v_1     3796 non-null   int64  
 6   v_8     3796 non-null   int64  
 7   v_12    3796 non-null   int64  
 8   v_25    3796 non-null   int64  
 9   v_20    3796 non-null   int64  
 10  v_18    3796 non-null   int64  
 11  v_3     3796 non-null   int64  
 12  v_11    3796 non-null   int64  
 13  v_14    3796 non-null   int64  
 14  v_26    3796 non-null   int64  
 15  v_27    3796 non-null   int64  
 16  v_30    3796 non-null   int64  
 17  v_9     3796 non-null   int64  
 18  v_31    3796 non-null   int64  
 19  v_28    3796 non-null   int64  
 20  v_13    3796 non-null   int64  
 21  v_33    3796 non-null   int64  
 22  

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x_train,y_label,test_size=0.2,random_state=42,stratify=y_label)

# Base model

In [12]:
from sklearn.dummy import DummyClassifier
base_clf = DummyClassifier(strategy='most_frequent')
base_clf.fit(X_train, y_train)
print('Training accuracy:{0:0.2f}'.format(base_clf.score(X_train, y_train)))
print('Testing accuracy:{0:0.2f}'.format(base_clf.score(X_test, y_test)))
print('Score: ', base_clf.score(X_train, y_train))

Training accuracy:0.49
Testing accuracy:0.49
Score:  0.49077733860342554


# Different models

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score,f1_score,roc_curve,ConfusionMatrixDisplay


In [14]:
models = {
     "Logistic Regression":LogisticRegression(),
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "KNear Neighbor":KNeighborsClassifier(),
    "SVM":SVC(),
    "Bagging":BaggingClassifier(),
    "AdaBoost":AdaBoostClassifier(),
    "Gradient Boost": GradientBoostingClassifier()
    
    
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    model_train_f1 = f1_score(y_train,y_train_pred,average='weighted')
   # model_train_accuracy = accuracy_score(y_train,y_train_pred)
   # model_train_precision = precision_score(y_train,y_train_pred,average = 'weighted') 
  # model_train_recall = recall_score(y_train,y_train_pred,average='weighted')
    
    model_test_f1 = f1_score(y_test,y_test_pred,average='weighted')

    # model_test_accuracy = accuracy_score(y_test,y_test_pred)
    # model_test_precision = precision_score(y_test,y_test_pred,average='weighted')
    # model_test_recall = recall_score(y_test,y_test_pred,average='weighted')
    
    print(list(models.keys())[i])
    
    print("Model performance for trianing set")
    
    # print("Accuracy:{:.4f}".format(model_train_accuracy))
    print("F1 score:{:.4f}".format(model_train_f1))
    # print("Precision:{:.4f}".format(model_train_precision))
    # print("Recall:{:.4f}".format(model_train_recall))
    
    print("Model performance for test set")
    
    # print("Accuracy:{:.4f}".format(model_test_accuracy))
    print("F1 score:{:.4f}".format(model_test_f1))
    # print("Precision:{:.4f}".format(model_test_precision))
    # print("Recall:{:.4f}".format(model_test_recall))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Logistic Regression
Model performance for trianing set
F1 score:0.5864
Model performance for test set
F1 score:0.5794
Decision Tree
Model performance for trianing set
F1 score:1.0000
Model performance for test set
F1 score:0.6364
Random Forest
Model performance for trianing set
F1 score:1.0000
Model performance for test set
F1 score:0.7245
KNear Neighbor
Model performance for trianing set
F1 score:0.6007
Model performance for test set
F1 score:0.4268
SVM
Model performance for trianing set
F1 score:0.3231
Model performance for test set
F1 score:0.3231
Bagging
Model performance for trianing set
F1 score:0.9861
Model performance for test set
F1 score:0.6963
AdaBoost
Model performance for trianing set
F1 score:0.7285
Model performance for test set
F1 score:0.6869
Gradient Boost
Model performance for trianing set
F1 score:0.8315
Model performance for test set
F1 score:0.7353


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3036, 42)
(760, 42)
(3036,)
(760,)


In [17]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [18]:
prediction =model.predict(X_test)

In [19]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(prediction,y_test))
print(confusion_matrix(y_test,prediction))
print(accuracy_score(prediction,y_test))

              precision    recall  f1-score   support

           0       0.71      0.78      0.75       227
           1       0.94      0.75      0.84       465
           2       0.30      0.62      0.41        68

    accuracy                           0.75       760
   macro avg       0.65      0.72      0.66       760
weighted avg       0.82      0.75      0.77       760

[[178  56  15]
 [ 11 351  11]
 [ 38  58  42]]
0.7513157894736842


In [20]:
test_data = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [21]:
prediction = model.predict(test_data)

In [22]:
final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})

In [23]:
final_prediction.to_csv('second_submission.csv',index=False)

In [24]:
final_prediction.head(10)

,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,0
6,6,0
7,7,2
8,8,1
9,9,0


# Handling imbalanced data (Oversampling technique)

In [25]:
X = train_data.drop(['label'],axis=1)
y = train_data['label']

In [26]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape: ', Counter(y))
print('Resample data shape: ', Counter(y_ros))

Original dataset shape:  Counter({1: 1863, 0: 1243, 2: 690})
Resample data shape:  Counter({1: 1863, 2: 1863, 0: 1863})


In [27]:
X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2,random_state=42)

In [28]:
y_train.value_counts()

2    1530
0    1493
1    1448
Name: label, dtype: int64

In [29]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [30]:
prediction =model.predict(X_test)

In [31]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(prediction,y_test))
print(confusion_matrix(y_test,prediction))
print(accuracy_score(prediction,y_test))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       354
           1       0.86      0.87      0.86       411
           2       0.91      0.86      0.88       353

    accuracy                           0.87      1118
   macro avg       0.87      0.87      0.87      1118
weighted avg       0.87      0.87      0.87      1118

[[314  37  19]
 [ 27 356  32]
 [ 13  18 302]]
0.8694096601073346


In [32]:
test_data = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [33]:
prediction = model.predict(test_data)

In [34]:
final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})

In [35]:
final_prediction.to_csv('Third_submission.csv',index=False)

# Handling imbalanced data (Undersampling technique)

In [36]:
from imblearn.under_sampling import RandomUnderSampler

X = train_data.drop(['label'],axis=1)
y = train_data['label']

In [37]:
from collections import Counter
undersample = RandomUnderSampler(random_state = 0)
X_us, y_us = undersample.fit_resample(X, y)

print('Original dataset shape: ', Counter(y))
print('Original dataset shape: ', Counter(y_us))

Original dataset shape:  Counter({1: 1863, 0: 1243, 2: 690})
Original dataset shape:  Counter({0: 690, 1: 690, 2: 690})


In [38]:
X_train,X_test,y_train,y_test = train_test_split(X_us,y_us,test_size=0.2,random_state=42)
model = RandomForestClassifier()
model.fit(X_train,y_train)


RandomForestClassifier()

In [39]:
prediction =model.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(prediction,y_test))
print(confusion_matrix(y_test,prediction))
print(accuracy_score(prediction,y_test))

              precision    recall  f1-score   support

           0       0.66      0.74      0.70       128
           1       0.76      0.67      0.71       145
           2       0.57      0.57      0.57       141

    accuracy                           0.66       414
   macro avg       0.66      0.66      0.66       414
weighted avg       0.66      0.66      0.66       414

[[95 15 35]
 [ 5 97 25]
 [28 33 81]]
0.6594202898550725


In [40]:
test_data = pd.read_csv("../input/students-drop-out-prediction/test.csv")
prediction = model.predict(test_data)
final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})
final_prediction.to_csv('Third_submission.csv',index=False)

# SMOTE

In [41]:
from imblearn.over_sampling import SMOTE

X = train_data.drop(['label'],axis=1)
y = train_data['label']
oversample = SMOTE()
X_sm, y_sm = oversample.fit_resample(X, y)
counter = Counter(y_sm)
counter

Counter({1: 1863, 2: 1863, 0: 1863})

In [42]:
X_train,X_test,y_train,y_test = train_test_split(X_sm,y_sm,test_size=0.2,random_state=42)
model = RandomForestClassifier()
model.fit(X_train,y_train)
prediction =model.predict(X_test)

In [43]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(prediction,y_test))
print(confusion_matrix(y_test,prediction))
print(accuracy_score(prediction,y_test))

              precision    recall  f1-score   support

           0       0.75      0.85      0.80       328
           1       0.87      0.81      0.84       442
           2       0.80      0.76      0.78       348

    accuracy                           0.81      1118
   macro avg       0.81      0.81      0.81      1118
weighted avg       0.81      0.81      0.81      1118

[[279  41  50]
 [ 23 360  32]
 [ 26  41 266]]
0.8094812164579607


In [44]:
test_data = pd.read_csv("../input/students-drop-out-prediction/test.csv")
prediction = model.predict(test_data)
final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})
final_prediction.to_csv('Third_submission.csv',index=False)

# Feature Selection

In [45]:
train= pd.read_csv("../input/students-drop-out-prediction/train.csv",index_col='id')
test = pd.read_csv("../input/students-drop-out-prediction/test.csv",index_col='id')

In [46]:
print("Shape of train data:",train.shape)
print("Shape of test data:",test.shape)

train.head()

train.info() 

x = train.drop(columns='label')
y = train.label

from sklearn.feature_selection import SelectKBest, mutual_info_regression
skb = SelectKBest(mutual_info_regression, k = 15)
X_new = skb.fit_transform(x,y)

print(f'Shape of feature matrix after feature selection:{X_new.shape}')

select_features=skb.get_feature_names_out()

print(select_features)


Shape of train data: (3796, 42)
Shape of test data: (1628, 41)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3796 entries, 0 to 3795
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v_21    3796 non-null   int64  
 1   v_32    3796 non-null   int64  
 2   v_15    3796 non-null   int64  
 3   v_4     3796 non-null   int64  
 4   v_1     3796 non-null   int64  
 5   v_8     3796 non-null   int64  
 6   v_12    3796 non-null   int64  
 7   v_25    3796 non-null   int64  
 8   v_20    3796 non-null   int64  
 9   v_18    3796 non-null   int64  
 10  v_3     3796 non-null   int64  
 11  v_11    3796 non-null   int64  
 12  v_14    3796 non-null   int64  
 13  v_26    3796 non-null   int64  
 14  v_27    3796 non-null   int64  
 15  v_30    3796 non-null   int64  
 16  v_9     3796 non-null   int64  
 17  v_31    3796 non-null   int64  
 18  v_28    3796 non-null   int64  
 19  v_13    3796 non-null   int64  
 20  v_33    379

In [47]:
x_test_new = test_data[select_features]

In [48]:
x_test_new.head()

,v_32,v_4,v_25,v_18,v_30,v_31,v_33,v_17,v_19,v_29,v_10,v_5,v_6,v_22,v_38
0,90,121,114,108,1,28,37,51,28,27,27,48,21,25,23
1,123,126,3,261,1,45,35,60,25,26,35,41,20,32,27
2,6,102,181,103,1,33,29,49,24,19,25,39,13,25,54
3,6,148,110,103,0,28,29,49,24,21,25,38,14,27,12
4,6,148,1,12,0,29,28,49,22,17,24,36,13,25,18


In [49]:
X_data =pd.DataFrame(data=X_new,columns=skb.get_feature_names_out())

X_data.head()

,v_32,v_4,v_25,v_18,v_30,v_31,v_33,v_17,v_19,v_29,v_10,v_5,v_6,v_22,v_38
0,6.0,55.0,5.0,30.0,0.0,26.0,29.0,48.0,24.0,22.0,25.0,37.0,14.0,28.0,48.0
1,94.0,65.0,4.0,199.0,1.0,27.0,30.0,55.0,27.0,18.0,26.0,46.0,13.0,27.0,53.0
2,166.0,155.0,1.0,30.0,1.0,31.0,29.0,48.0,24.0,19.0,25.0,37.0,12.0,25.0,43.0
3,6.0,121.0,1.0,23.0,0.0,26.0,30.0,48.0,25.0,19.0,27.0,40.0,15.0,25.0,43.0
4,51.0,21.0,118.0,106.0,1.0,49.0,41.0,61.0,41.0,29.0,32.0,50.0,27.0,30.0,24.0


In [50]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_data,y,test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

y_train.unique()

y_train.value_counts()


model = RandomForestClassifier()
model.fit(X_train,y_train)


(3036, 15)
(760, 15)
(3036,)
(760,)


RandomForestClassifier()

In [51]:
prediction =model.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(prediction,y_test))
print(confusion_matrix(y_test,prediction))
print(accuracy_score(prediction,y_test))

prediction = model.predict(x_test_new)

final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})

final_prediction

              precision    recall  f1-score   support

           0       0.72      0.79      0.75       222
           1       0.91      0.75      0.82       464
           2       0.30      0.55      0.39        74

    accuracy                           0.74       760
   macro avg       0.64      0.70      0.65       760
weighted avg       0.79      0.74      0.76       760

[[175  53  16]
 [ 18 346  17]
 [ 29  65  41]]
0.7394736842105263


,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1623,1623,0
1624,1624,1
1625,1625,0
1626,1626,2


In [52]:
Binary_columns = ['v_1','v_26','v_11','v_14','v_30','v_28','v_9','v_27']
Nominal_columns = ['v_32','v_4','v_3','v_20','v_21','v_18','v_25','v_12']
Ordinal_columns =['v_31','v_15','v_19','v_13','v_33','v_17','v_29','v_23','v_6','v_24','v_10','v_5','v_22','v_0']
Ratio_columns =['v_16','v_2','v_8','v_7','v_34','v_35','v_36','v_37','v_38','v_39','v_40']

In [53]:
train = train_data.drop(columns='label')
label = train_data['label']

In [54]:
frames = [train,test_data]
total_data = pd.concat(frames)
total_data.shape


(5424, 42)

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,OrdinalEncoder

ct = ColumnTransformer([('scaler', MinMaxScaler(),Ratio_columns),
                        ('ordinal', OrdinalEncoder(),Ordinal_columns),
                        ('encoder', OneHotEncoder(sparse=False),Nominal_columns)])
total_transformed_data=ct.fit_transform(total_data)

ct

ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                 ['v_16', 'v_2', 'v_8', 'v_7', 'v_34', 'v_35',
                                  'v_36', 'v_37', 'v_38', 'v_39', 'v_40']),
                                ('ordinal', OrdinalEncoder(),
                                 ['v_31', 'v_15', 'v_19', 'v_13', 'v_33',
                                  'v_17', 'v_29', 'v_23', 'v_6', 'v_24', 'v_10',
                                  'v_5', 'v_22', 'v_0']),
                                ('encoder', OneHotEncoder(sparse=False),
                                 ['v_32', 'v_4', 'v_3', 'v_20', 'v_21', 'v_18',
                                  'v_25', 'v_12'])])

In [56]:
print(total_transformed_data.shape)

(5424, 728)


In [57]:
X_train = total_transformed_data[:3796,:]
X_test = total_transformed_data[3796:,:]
print(X_train.shape)
print(X_test.shape)

(3796, 728)
(1628, 728)


In [58]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_train,label,test_size=0.2,random_state=42,stratify=label)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3036, 728)
(760, 728)
(3036,)
(760,)


In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
np.random.seed(42)
g2=GridSearchCV(RandomForestClassifier(random_state=42),{'n_estimators':[10,20,50],'max_features':['auto','sqrt'],'max_depth':[2,4],'min_samples_split':[2,5],'min_samples_leaf':[1,2],'bootstrap':[True,False]},cv=10)
g2.fit(x_train,y_train)
g2.best_params_

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 20}

In [60]:
r=RandomForestClassifier(n_estimators=20,max_depth=4,max_features='auto',min_samples_split=5,min_samples_leaf=1,bootstrap=True,random_state=42)
r.fit(x_train, y_train)
print(r.score(x_train,y_train))

print(r.score(x_test,y_test))

0.6821475625823452
0.6802631578947368


In [61]:
from sklearn.ensemble import AdaBoostClassifier
a=AdaBoostClassifier(n_estimators=20,base_estimator=r,random_state=42)
a.fit(x_train,y_train)
print(a.score(x_train,y_train))
print(a.score(x_test,y_test))
print(classification_report(a.predict(x_test),y_test))

0.8138998682476943
0.718421052631579
              precision    recall  f1-score   support

           0       0.71      0.72      0.71       245
           1       0.90      0.76      0.82       439
           2       0.26      0.47      0.34        76

    accuracy                           0.72       760
   macro avg       0.62      0.65      0.62       760
weighted avg       0.77      0.72      0.74       760



In [62]:
from sklearn.ensemble import BaggingClassifier
b=BaggingClassifier(base_estimator=r,random_state=42)
b.fit(x_train,y_train)
print(b.score(x_train,y_train))
print(b.score(x_test,y_test))
print(classification_report(a.predict(x_test),y_test))

0.6525032938076416
0.6381578947368421
              precision    recall  f1-score   support

           0       0.71      0.72      0.71       245
           1       0.90      0.76      0.82       439
           2       0.26      0.47      0.34        76

    accuracy                           0.72       760
   macro avg       0.62      0.65      0.62       760
weighted avg       0.77      0.72      0.74       760



In [63]:
prediction = a.predict(X_test)

In [64]:
final_prediction = pd.DataFrame({'id':test_data.id,'label':prediction})
final_prediction.to_csv('fourth_submission.csv',index=False)